# Implementación de Batching

**Objetivo:** Crear una función generadora que agrupe una lista de datos (documentos) en lotes (batches) de un tamaño especificado. Probar el generador con el dataset 20 Newsgroups.

In [1]:
import time
from sklearn.datasets import fetch_20newsgroups

In [2]:
start_time = time.time()
newsgroups_data = fetch_20newsgroups(subset='all',
                                     remove=('headers', 'footers', 'quotes'))
end_time = time.time()
print(f"Dataset cargado en {end_time - start_time:.2f} segundos.")

documents = newsgroups_data.data
num_docs = len(documents)
print(f"Número total de documentos: {num_docs}")

del newsgroups_data

Dataset cargado en 1.24 segundos.
Número total de documentos: 18846


In [3]:
def batch_generator(data_list, batch_size):
    """
    Generador que divide una lista de datos en lotes (batches).

    Args:
        data_list (list): La lista completa de datos a dividir.
        batch_size (int): El tamaño deseado para cada lote.

    Yields:
        list: Un lote  de datos. El último lote puede ser más pequeño.
              No produce nada si la lista de entrada está vacía o batch_size <= 0.
    """
    if not data_list or batch_size <= 0:
        print("Advertencia: La lista de datos está vacía o el tamaño de lote es inválido. No se generarán lotes.")
        return

    num_total_items = len(data_list)
    # Iteramos sobre la lista en pasos de tamaño batch_size
    for start_index in range(0, num_total_items, batch_size):
        end_index = start_index + batch_size
        batch = data_list[start_index:end_index]
        yield batch

In [7]:
batch_size = 4096
batch_gen = batch_generator(documents, batch_size)

num_batches_yielded = 0
first_batch_sample = None
total_items_in_batches = 0

# Iteramos sobre el generador para obtener los lotes
for i, current_batch in enumerate(batch_gen):
    num_batches_yielded += 1
    batch_len = len(current_batch)
    total_items_in_batches += batch_len
    # Realizamos la impresion de los primeros elementos del primer lote, luego mostramos solamente su tamaño
    if i == 0:
        first_batch_sample = current_batch
        print(f"  Lote {i+1}: Tamaño = {batch_len}")
        print("    Primeros elementos (inicio):")
        for j, item in enumerate(current_batch[:3]):
             print(f"      {j+1}: '{item[:80]}...'")
    elif i > 0:
        print(f"  Lote {i+1}: Tamaño = {batch_len}")

# Calcular cuántos lotes esperábamos
expected_batches = (num_docs + batch_size - 1) // batch_size if num_docs > 0 and batch_size > 0 else 0

print("\n--- Resumen de la prueba del generador ---")
print(f"Tamaño de lote configurado: {batch_size}")
print(f"Número total de documentos: {num_docs}")
print(f"Número de lotes generados: {num_batches_yielded}")
print(f"Número esperado de lotes: {expected_batches}")
print(f"Total de ítems procesados en lotes: {total_items_in_batches}")

  Lote 1: Tamaño = 4096
    Primeros elementos (inicio):
      1: '

I am sure some bashers of Pens fans are pretty confused about the lack
of any ...'
      2: 'My brother is in the market for a high-performance video card that supports
VESA...'
      3: '



	Finally you said what you dream about. Mediterranean???? That was new....
	...'
  Lote 2: Tamaño = 4096
  Lote 3: Tamaño = 4096
  Lote 4: Tamaño = 4096
  Lote 5: Tamaño = 2462

--- Resumen de la prueba del generador ---
Tamaño de lote configurado: 4096
Número total de documentos: 18846
Número de lotes generados: 5
Número esperado de lotes: 5
Total de ítems procesados en lotes: 18846
